In [1]:
from bs4 import BeautifulSoup
from selenium  import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.touch_actions import TouchActions
import time
import pandas as pd
driver = webdriver.Chrome(ChromeDriverManager().install())




Checking for linux64 chromedriver:76.0.3809.126 in cache
Driver found in /home/shakti/.wdm/chromedriver/76.0.3809.126/linux64/chromedriver


In [2]:
driver.get('https://www.prokabaddi.com')
time.sleep(1)
standings=driver.find_element_by_link_text("Standings")
standings.click()
#### TO WRITE DRIVER PAGE AS HTML
#soup = BeautifulSoup(driver.page_source,"html.parser")
# with open("StandingsPage1.html", "w") as file:
#     file.write(str(soup))
time.sleep(2)
tableCOntent=driver.find_element_by_class_name("sipk-standing-page ")
table=tableCOntent.text
table=table.split('\n')[1:]
numberOfColumns=8
numberOfTeams=12
columns=table[:numberOfColumns]
tablevalues=[]
for i in range(numberOfColumns,len(table),numberOfColumns):
    tablevalues.append(table[i:i+numberOfColumns])
tablevalues=tablevalues[:numberOfTeams]
LeagueTable=pd.DataFrame.from_records(tablevalues,columns=columns)
LeagueTable.to_csv("./Leaguetable.csv")

In [111]:
# from selenium.webdriver.common.keys import Keys 
# driver.get('https://www.prokabaddi.com')
# time.sleep(1)
# Schedule=driver.find_element_by_class_name("schedule-fixtures-results")
# time.sleep(1)
# driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
# time.sleep(1)
# # table=driver.find_element_by_class_name("more-btn")
# # table.click()
# # #### TO WRITE DRIVER PAGE AS HTML
# soup = BeautifulSoup(driver.page_source,"html.parser")
# with open("SchedulePage.html", "w") as file:
#     file.write(str(soup))

# table=driver.find_element_by_class_name("si-rhs-matNumber")

''

IMPORTANT GUIDE FOR SELENIUM:
https://www.pluralsight.com/guides/web-scraping-with-selenium

In [3]:
driver.get('https://www.flashscore.com.au/kabaddi/india/pro-kabaddi/results/')
time.sleep(1)
fixtureHistory=driver.find_element_by_id("live-table")
fixtureHistory=fixtureHistory.text.split('\n')[3:]
# #### TO WRITE DRIVER PAGE AS HTML
# soup = BeautifulSoup(driver.page_source,"html.parser")
# with open("ResultsPage.html", "w") as file:
#     file.write(str(soup))

columns=['Date','HomeTeam','AwayTeam','Total_Points_HomeTeam','Total_Points_AwayTeam','FirstHalf_Points_HomeTeam','FirstHalf_Points_AwayTeam','SecondHalf_Points_HomeTeam','SecondHalf_Points_AwayTeam']
tablevalues=[]
for i in range(0,len(fixtureHistory),len(columns)):
    tablevalues.append(fixtureHistory[i:i+len(columns)])
ResultsTable=pd.DataFrame.from_records(tablevalues,columns=columns)
ResultsTable.to_csv('./PointsTable.csv')

In [5]:
driver.get('https://www.flashscore.com.au/kabaddi/india/pro-kabaddi/fixtures/')
time.sleep(1)
fixtureUpcoming=driver.find_element_by_id("live-table").text.split('\n')[3:]
fixtureUpcoming
columns=['date','HomeTeam','AwayTeam','Drop1','Drop2']
tablevalues=[]
for i in range(0,len(fixtureUpcoming),len(columns)):
    tablevalues.append(fixtureUpcoming[i:i+len(columns)])
UpcomingTable=pd.DataFrame.from_records(tablevalues,columns=columns)
UpcomingTable=UpcomingTable.drop(['Drop1','Drop2'],axis=1)
UpcomingTable.to_csv('./Upcomingfixtures.csv')

# #### TO WRITE DRIVER PAGE AS HTML
# soup = BeautifulSoup(driver.page_source,"html.parser")
# with open("FixturesPage.html", "w") as file:
#     file.write(str(soup))

In [ ]:
driver.quit()